In [1]:
# !pip install pandas
# !pip install numpy

In [2]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None # Fuck warnings

<h1> Загрузка данных

In [3]:
import json


with open(r'C:\Users\User\Documents\snli_1.0/snli_1.0_train.jsonl', 'r') as json_file:
    json_list = list(json_file)

result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [4]:
df_train = pd.json_normalize(result)

In [5]:
df_cut = df_train[:10000]

In [6]:
df_cut.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse'],
      dtype='object')

In [7]:
df_cut.head(1)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...


<h1> Предобработка данных

<h3> Оставляем только основу от слов

In [8]:
import nltk.stem


my_stem = nltk.stem.SnowballStemmer('english')

def stemSentence(sentence):
    new_sentence = ''

    for word in sentence.split():
        new_sentence += my_stem.stem(word) + ' '
    
    return new_sentence[:-1]

<h3> Убираем символы

In [9]:
import string

def deleteSymbs(sentence):
    res = ''

    for char in sentence:
        if char not in string.punctuation:
            res += char
    
    return res

<h3> Agregate function

In [10]:
def fixSentence(sentence):
    for func in ['deleteSymbs', 'stemSentence']:
        sentence = eval(func)(sentence)
    
    return sentence

In [11]:
for i in [1, 2]:
    df_cut[f'Fixed_sentence{i}'] = df_cut[f'sentence{i}'].apply(fixSentence)

<h1> Word2Vec

In [23]:
texts = np.concatenate((df_cut.Fixed_sentence1.values, df_cut.Fixed_sentence2.values))

In [32]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api


corpus = api.load('text8')  # download the corpus and return it opened as an iterable
model = Word2Vec(corpus)    # train a model from the corpus
model.most_similar("car")

[==================================================] 100.0% 31.6/31.6MB downloaded
<ipython-input-32-14c79032dbe6>:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("car")


[('driver', 0.8082566857337952),
 ('cars', 0.7426753044128418),
 ('motorcycle', 0.7394026517868042),
 ('truck', 0.707937479019165),
 ('vehicle', 0.6957083344459534),
 ('taxi', 0.6873030066490173),
 ('racing', 0.6870726943016052),
 ('passenger', 0.6720129251480103),
 ('mustang', 0.656194269657135),
 ('automobile', 0.6460007429122925)]

In [35]:
# model = api.load('glove-twitter-200') # 758MB
model = api.load('glove-wiki-gigaword-100') # 128MB - гораздо адекватнее

[==================================================] 100.0% 128.1/128.1MB downloaded


In [42]:
model.most_similar('horse')

[('horses', 0.8550404906272888),
 ('riding', 0.7131650447845459),
 ('dog', 0.7109653353691101),
 ('racing', 0.6629325151443481),
 ('bike', 0.6614626049995422),
 ('thoroughbred', 0.6600428819656372),
 ('bull', 0.6456228494644165),
 ('jockey', 0.6354342699050903),
 ('breeders', 0.6333900690078735),
 ('race', 0.6270371675491333)]